In [5]:
import numpy as np
import os

import plotly.graph_objs as go

from trProcess import *

In [7]:
tracenum = 10
baselinelist = {
    "FQ": "FqCoDel_0_500ms1200", 
    "FIFO": "PfifoFast_0_1200",
    "DRR": "Dwrr_9_1.00",
    "Confucius": "Auto_0_1.00_ExpClass4.00"
}

indir = "ns-3.34/logs/"

numflows = [5, 10, 20, 40, 60, 80, 100]
appPlot, fctPlot = [], []
maxFct, maxApp = {}, {}
for name, baseline in baselinelist.items():
    maxFct[name], maxApp[name] = [], []
    for numflow in numflows:
        exp = "46_Video1_Web15%03d" % numflow
        maxFctTrace, maxAppTrace = [], []
        for traceid in range(tracenum):
            trace = "maxwell_20.0_0.01_300_%d.pitree-trace" % traceid
            conf = os.path.join (indir, exp, baseline, trace)
                
            bws = []
            curMaxApp, curAppDur, _, _ = appTraceProcessor (os.path.join (conf, 'app.tr'))
            curMaxAppFlows = 0
            for port, appDelay in curAppDur.items():
                if port in range(10000, 20000):
                    curMaxAppFlows = max(curMaxAppFlows, appDelay)
            maxAppTrace.append(curMaxAppFlows)
    
            curAvgFct, curMaxFct, fctLastStart, fctFirstEnd, fcts = fctTraceProcessor(os.path.join (conf, 'fct.tr'))
            maxFctTrace.append(curMaxFct)
            bws = np.array(bws)
        maxFct[name].append(np.average(maxFctTrace))
        maxApp[name].append(np.average(maxAppTrace))
        # print(np.average(maxAppTrace), np.average(maxFctTrace), baseline, numflow)

print ("VideoDelayDegradation ......")
tmpStr = ""
for name, baseline in baselinelist.items():
    tmpStr += "\t%s" % name
print (tmpStr)
for idx in range(len(numflows)):
    tmpStr = "%d" % numflows[idx]
    for name, baseline in baselinelist.items():
        tmpStr += "\t%d" % maxApp[name][idx]
    print (tmpStr)

print ("PageLoadingTime ......")
tmpStr = ""
for name, baseline in baselinelist.items():
    tmpStr += "\t%s" % name
print (tmpStr)
for idx in range(len(numflows)):
    tmpStr = "%d" % numflows[idx]
    for name, baseline in baselinelist.items():
        tmpStr += "\t%d" % (maxFct[name][idx] - maxFct["FQ"][idx])
    print (tmpStr)


for name, baseline in baselinelist.items():                
    appPlot.append(go.Scatter(
        x=numflows,
        y=maxApp[name],
        mode="markers+lines",
        name=name
    ))             
    fctPlot.append(go.Scatter(
        x=numflows,
        y=np.array(maxFct[name]) - np.array(maxFct["FQ"]),
        mode="markers+lines",
        name=name
    ))

fig = go.Figure(data=appPlot)
fig.update_layout(
    title="46_Video1_Web15n",
    xaxis_title="Number of flows",
    yaxis_title="Vidoe delay degradation"
)
fig.show()
  
fig = go.Figure(data=fctPlot)
fig.update_layout(
    title="46_Video1_Web15n",
    xaxis_title="Number of flows",
    yaxis_title="FCT Difference"
)
fig.show()

VideoDelayDegradation ......
	FQ	FIFO	DRR	Confucius
5	0	0	0	0
10	0	0	0	0
20	0	0	0	0
40	282	248	0	0
60	556	540	0	0
80	726	696	0	0
100	874	854	0	0
PageLoadingTime ......
	FQ	FIFO	DRR	Confucius
5	0	15	13	6
10	0	28	45	39
20	0	28	95	79
40	0	32	189	157
60	0	14	237	189
80	0	24	344	241
100	0	28	418	261
